# Understanding Encapsulation in Object-Oriented Programming

## What is Encapsulation?

Encapsulation is the OOP principle of bundling data (attributes) and methods (functions) that operate on that data into a single unit (a class), while restricting direct access to some of the object's components. Essentially, encapsulation prevents external code from being concerned with the internal workings of an object.

Encapsulation is easily confused with abstraction but as Grady Booch states in Object Oriented Analysis and Design:
> Abstraction and encapsulation are complementary concepts: abstraction focuses on the observable behavior of an object... encapsulation focuses upon the implementation that gives rise to this behavior... encapsulation is most often achieved through information hiding, which is the process of hiding all of the secrets of object that do not contribute to its essential characteristics.

Think of it like a medicine capsule:
- The capsule shell protects the medicine inside
- You take the capsule whole - you don't access the medicine directly
- The capsule controls how the medicine is released

## The 3 Levels of Access Control
1. Public Access (attribute)
```python

class BankAccount:
    def __init__(self):
        self.balance = 1000  # Public - anyone can read/write
```
Problem: Anyone can do account.balance = -5000 or account.balance = "not a number"
2. Protected Access (_attribute)
```python

class BankAccount:
    def __init__(self):
        self._balance = 1000  # Protected - convention only
```

Python Convention: A single underscore means "Hey, this is internal, please don't touch from outside the class"

3. Private Access (__attribute)
```python

class BankAccount:
    def __init__(self):
        self.__balance = 1000  # Name mangling makes it harder to access
```
Python Reality: __balance becomes _BankAccount__balance - not truly private, but harder to access accidentally

## Why Encapsulation Matters: Real-World Examples
### Bank Account Without Encapsulation
```python

# DANGEROUS: Direct attribute access
class BankAccount:
    def __init__(self):
        self.balance = 0

account = BankAccount()
account.balance = -1000000  # Negative balance!
account.balance = "hello"   # Not even a number!
```
### Bank Account With Encapsulation
```python

# SAFE: Controlled access via properties
class BankAccount:
    def __init__(self):
        self._balance = 0

    # getter
    def get_balance(self):
        pass
    # setter
    def set_balance(self, value):
        self._balance = value
    
    @property
    def balance(self):
        return self._balance
    
    def deposit(self, amount):
        try:
            float_amount = float(amount)
            if float_amount > 0:
                self._balance += float_amount
        except ValueError:
            print('not integer or float or compatible string')
            
    def withdraw(self, amount):
        if 0 < amount <= self._balance:
            self._balance -= amount

account = BankAccount()
account.deposit(1000)   # ✅ Controlled
account.withdraw(500)   # ✅ Validated
# account.balance = -1000  # ❌ Can't do this - AttributeError!
```

## How Different Languages Handle Privacy
### Java (Strict Enforcement)
```java

public class BankAccount {
    private double balance;  // COMPILE-TIME ENFORCEMENT
    
    public double getBalance() {  // Must use getter
        return this.balance;
    }
    
    public void setBalance(double amount) {  // Must use setter
        if (amount >= 0) {
            this.balance = amount;
        }
    }
}
```
Key: private means compiler-enforced privacy - you literally cannot access it from outside the class.
### C++ (Also Strict)
```c++

class BankAccount {
private:  // Explicit private section
    double balance;
    
public:
    double getBalance() { return balance; }
};
```

### Python (Convention-Based)
```python

class BankAccount:
    def __init__(self):
        self._balance = 0  # CONVENTION-BASED "protected"
        self.__secret = 42  # Name-mangled "private"
    
    def get_balance(self):
        return self._balance
```
Python Philosophy: "We're all consenting adults here" - trust developers to respect conventions.


## The Python Way: Properties for Encapsulation
### Step 1: Basic Property
```python

class Temperature:
    def __init__(self):
        self._celsius = 0
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:  # Absolute zero
            raise ValueError("Temperature too low")
        self._celsius = value
```
### Step 2: Computed Properties
```python

class Temperature:
    def __init__(self):
        self._celsius = 0
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Temperature too low")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return (self._celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        self._celsius = (value - 32) * 5/9
```
## Real-World Benefits of Encapsulation
### 1. Data Validation
```python

class User:
    def __init__(self):
        self._email = None
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        if "@" not in value:
            raise ValueError("Invalid email")
        self._email = value
```

## 2. Lazy Loading
```python

class DatabaseConnection:
    def __init__(self):
        self._connection = None
    
    @property
    def connection(self):
        if self._connection is None:
            print("Creating expensive database connection...")
            self._connection = "Connected to DB"
        return self._connection
```
## 3. Access Logging
```python

class SecureData:
    def __init__(self):
        self._data = "Sensitive Info"
        self._access_count = 0
    
    @property
    def data(self):
        self._access_count += 1
        print(f"Data accessed {self._access_count} times")
        return self._data
```

## Wrup-up

| Situation	 | Recommended Approach |
| ---------  | -------------------- |
|Truly private data	| __attribute (name mangling)|
|Internal implementation |	_attribute (single underscore)|
|Public API with validation	| @property decorators|
|Read-only data	| Property with  getter only|
|Methods for internal use |	_method_name()|

### Pattern 1: The "Private" Pattern
```python

class SecretKeeper:
    def __init__(self):
        self.__secret = "I'm really private"
    
    def reveal(self, password):
        if password == "open sesame":
            return self.__secret
        return "Access denied"
```

### Pattern 2: The "Protected" Pattern
```python

class Parent:
    def __init__(self):
        self._protected_data = "For family only"

class Child(Parent):
    def use_parent_data(self):
        return self._protected_data  # Child can access
```

### Pattern 3: The "Property" Pattern
```python

class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    @property
    def area(self):
        return self._width * self._height
    
    @property 
    def perimeter(self):
        return 2 * (self._width + self._height)
```

### Why Python's Approach is Different
Philosophical Differences
| Language |	Philosophy |	Privacy Approach |
| - | - | - |
| Java/C++ |	"Trust the compiler" |	Compiler enforcement |
| Python |	"Trust the programmer" |	Convention + name mangling |
| JavaScript |	"Everything is public" |	Closures for privacy |

### Python's Practical Reality
```python 

class Example:
    def __init__(self):
        self.public = 1      # Anyone can access
        self._protected = 2  # Please don't touch
        self.__private = 3   # Harder to access
    
    def show_private(self):
        return self.__private  # Works inside class

obj = Example()
print(obj.public)           # ✅ 1
print(obj._protected)       # ⚠️ 2 (works but discouraged)
# print(obj.__private)      # ❌ AttributeError
print(obj._Example__private) # ⚠️ 3 (works but don't do this!)
```

### Best Practices Summary

- Use _single_leading_underscore for "protected" attributes
- Use @property decorators for controlled access with validation
- Use __double_underscore sparingly - mainly to avoid name clashes in inheritance
- Document your public API clearly so users know what's safe to use
- Test your encapsulation to ensure it actually protects what you intend

### The Big Picture

Encapsulation in Python isn't about preventing access (like in Java), but about communicating intent and providing safe interfaces.

Think of it as putting up signs rather than walls:

    public_attribute = "Public park - come on in!"

    _protected_attribute = "Staff only - please use main entrance"

    __private_attribute = "Maintenance access - serious consequences if you enter"

This approach aligns with Python's philosophy of readability and practicality over strict enforcement!

# Background info for the exercise (Hashing)

## Hashing a password
you will see it in the proper course but let us understand the principle to continue with our example

simple explaination video for reference: https://www.youtube.com/watch?v=deNIgVyDyJY

Hasing is not encrypting, it's a one-way mathematical transformation
```
password123 --> 5f4dcc3b5aa765d61d8327deb882cf99
```
Imagine you put an apple, a banana, and some grapes into a high-powered blender. 
- Ingredients (Input): This is your original, plain-text password (`password123`)
- The Blender (Hash Function): This is a specific, complex mathematical algorithm that mixes the ingredients in a unique and complex way.
- The Smoothie (Output): This is the resulting fixed-length string of characters, (e.g., `5f4dcc3b5aa765d61d8327deb882cf99`)

Ideally a little change in the input became a big change in the output

# Improved DocumentManager example

In [ ]:
import hashlib

class User:
    """
    User class with encapsulated attributes and password hashing.
    """
    
    # Class-level constant for valid roles
    VALID_ROLES = {'user', 'administrator'}
    
    def __init__(self, username: str, role: str, password: str):
        """
        Initialize a new User.
        
        Parameters
        ----------
        username : str
            Unique username
        role : str
            User role (must be 'user' or 'administrator')
        password : str
            Plain text password (will be hashed)
        """
        self._username = username
        self.set_role(role) # we use the setter to validate
        self._password_hash = self._hash_password(password)
    
    # ----- Property Getters -----
    @property
    def username(self) -> str:
        """Get the username (read-only)."""
        return self._username
    
    @property
    def role(self) -> str:
        """Get the user role."""
        return self._role
    
    @property
    def password_hash(self) -> str:
        """Get the password hash (for storage purposes)."""
        return self._password_hash
    
    # ----- Property Setters -----
    def set_role(self, new_role: str) -> None:
        """
        Set a new role for the user with validation.
        
        Parameters
        ----------
        new_role : str
            New role (must be 'user' or 'administrator')
            
        Raises
        ------
        ValueError
            If role is not valid
        """
        if new_role not in self.VALID_ROLES:
            raise ValueError(f"Invalid role. Must be one of: {self.VALID_ROLES}")
        self._role = new_role
    
    def change_password(self, old_password: str, new_password: str) -> bool:
        """
        Change user's password after verifying old password.
        
        Parameters
        ----------
        old_password : str
            Current password for verification
        new_password : str
            New password to set
            
        Returns
        -------
        bool
            True if password was changed, False if old password incorrect
        """
        if not self.authenticate(old_password):
            return False
        self._password_hash = self._hash_password(new_password)
        return True
    
    # ----- Private Methods -----
    def _hash_password(self, password: str) -> str:
        """
        Hash a password using SHA-256 (without salt).
        
        Parameters
        ----------
        password : str
            Plain text password
            
        Returns
        -------
        str
            Hashed password as hex string
        """
        # WARNING: Hashing without salt is not secure for production
        return hashlib.sha256(password.encode()).hexdigest()
    
    # ----- Public Methods -----
    def authenticate(self, password: str) -> bool:
        """
        Authenticate user with password.
        
        Returns
        -------
        bool
            True if authentication successful, False otherwise
        """ 
        if not isinstance(password, str):
            return False
            
        # Compare hashed input with stored hash
        input_hash = self._hash_password(password)
        return self._password_hash == input_hash
        
    def can_modify(self, document: 'Document') -> bool:
        """Check if user can modify a document"""        
        # Administrators can modify everything
        if self._role == 'administrator':
            return True
        
        # Users can modify their own documents
        if document.owner == self:
            return True
        
        return False
    
    def to_dict(self) -> dict:
        """Convert to dictionary for backward compatibility."""
        return {
            "username": self._username,
            "role": self._role,
            "password_hash": self._password_hash
        }
    
    def __str__(self) -> str:
        return f"User({self._username}, {self._role})"
    
    def __eq__(self, other: object) -> bool:
        """Check if two User objects represent the same user."""
        if not isinstance(other, User):
            return False
        return self._username == other._username


In [ ]:
class Document:
    """
    Document class with encapsulated attributes.
    
    Examples
    --------
    >>> user = User("alice", "user", "password123")
    >>> doc = Document(1, "My Doc", "Content", user, "private")
    """
    
    VALID_VISIBILITY = {'private', 'public'}
    
    def __init__(self, doc_id: int, title: str, content: str, owner: User, visibility: str):
        self._id = doc_id
        self._title = title
        self._content = content
        self._owner = owner
        self.set_visibility(visibility)  # we use setter for validating the input
    
    # ----- Property Getters -----
    @property
    def id(self) -> int:
        """Get document ID (read-only)."""
        return self._id
    
    @property
    def title(self) -> str:
        """Get document title."""
        return self._title
    
    @property
    def content(self) -> str:
        """Get document content."""
        return self._content
    
    @property
    def owner(self) -> User:
        """Get document owner."""
        return self._owner
    
    @property
    def visibility(self) -> str:
        """Get document visibility."""
        return self._visibility
    
    # ----- Property Setters -----
    def set_title(self, new_title: str, user: User) -> bool:
        """
        Change document title if user has permission.
        
        Parameters
        ----------
        new_title : str
            New title
        user : User
            User attempting the change
            
        Returns
        -------
        bool
            True if change successful, False otherwise
        """
        if user.can_modify(self):
            self._title = new_title
            return True
        return False
    
    def set_content(self, new_content: str, user: User) -> bool:
        """
        Change document content if user has permission.
        
        Parameters
        ----------
        new_content : str
            New content
        user : User
            User attempting the change
            
        Returns
        -------
        bool
            True if change successful, False otherwise
        """
        if user.can_modify(self):
            self._content = new_content
            return True
        return False
    
    def set_visibility(self, new_visibility: str, user: User = None) -> bool:
        """
        Change document visibility if user has permission.
        
        Parameters
        ----------
        new_visibility : str
            New visibility setting
        user : User, optional
            User attempting the change. If None, assumes owner (for initialization)
            
        Returns
        -------
        bool
            True if change successful, False otherwise
        """
        # Allow setting without user check during initialization
        if user is None:
            if new_visibility not in self.VALID_VISIBILITY:
                raise ValueError(f"Invalid visibility. Must be one of: {self.VALID_VISIBILITY}")
            self._visibility = new_visibility
            return True
            
        # Normal permission check for subsequent changes
        if user.can_modify(self):
            if new_visibility not in self.VALID_VISIBILITY:
                raise ValueError(f"Invalid visibility. Must be one of: {self.VALID_VISIBILITY}")
            self._visibility = new_visibility
            return True
        return False
    
    def change_visibility(self, new_visibility: str, user: User) -> bool:
        """
        Deprecated: Use set_visibility instead.
        """
        return self.set_visibility(new_visibility, user)
    
    # ----- Public Methods -----
    def is_accessible_by(self, user: User) -> bool:
        """
        Check if a user can access (view) this document.
        
        Parameters
        ----------
        user : User
            User to check access for
            
        Returns
        -------
        bool
            True if user can access the document
        """
        return user.can_modify(self) or self._visibility == 'public'
    
    def to_dict(self) -> dict:
        """Convert to dictionary for backward compatibility."""
        return {
            "id": self._id,
            "title": self._title,
            "content": self._content,
            "owner": self._owner.username,  # Store username instead of object
            "visibility": self._visibility
        }
    
    def __str__(self) -> str:
        return f"Document({self._id}, '{self._title}', {self._visibility})"
    
    def __eq__(self, other: object) -> bool:
        """Check if two Document objects represent the same document."""
        if not isinstance(other, Document):
            return False
        return self._id == other._id





In [ ]:
class DocumentManager:
    """
    Document manager with encapsulated collections.
    """
    
    def __init__(self):
        # Initialize with private collections
        self._users = []
        self._documents = []
        self._current_user = None
        
        # Initialize with sample data
        self._initialize_sample_data()
    
    # ----- Property Getters -----
    @property
    def current_user(self) -> User:
        """Get the currently logged in user."""
        return self._current_user
    
    @property
    def user_count(self) -> int:
        """Get the number of registered users."""
        return len(self._users)
    
    @property
    def document_count(self) -> int:
        """Get the number of documents."""
        return len(self._documents)
    
    # ----- Private Methods -----
    def _initialize_sample_data(self) -> None:
        """Initialize with sample data."""
        # Create sample users
        admin = User("admin", "administrator", "admin")
        alice = User("alice", "user", "alice")
        bob = User("bob", "user", "bob")
        
        self._users.extend([admin, alice, bob])
        
        # Create sample documents
        self._documents.extend([
            Document(1, "Company Strategy", "Secret business plans...", admin, "private"),
            Document(2, "Project Report", "Q1 project results...", alice, "private"),
            Document(3, "Personal Notes", "My private thoughts...", alice, "private"),
            Document(4, "Public Info", "Company public information...", admin, "public")
        ])
    
    def _find_user_by_username(self, username: str) -> User:
        """
        Find a user by username.
        
        Parameters
        ----------
        username : str
            Username to search for
            
        Returns
        -------
        User or None
            User object if found, None otherwise
        """
        for user in self._users:
            if user.username == username:
                return user
        return None
    
    def _find_document_by_title(self, title: str) -> Document:
        """
        Find a document by title.
        
        Parameters
        ----------
        title : str
            Document title to search for
            
        Returns
        -------
        Document or None
            Document object if found, None otherwise
        """
        for doc in self._documents:
            if doc.title == title:
                return doc
        return None
    
    # ----- Public Methods -----
    def user_login(self, username: str, password: str) -> tuple:
        """
        Authenticate a user by username and password.
        
        Parameters
        ----------
        username : str
            The username to authenticate
        password : str
            The password provided by the user
            
        Returns
        -------
        tuple (User or None, bool)
            User object (if username exists) and authentication status
        """
        user = self._find_user_by_username(username)
        if user is None:
            return None, False
        
        if user.authenticate(password):
            self._current_user = user
            return user, True
        return user, False
    
    def user_logout(self) -> None:
        """Log out the current user."""
        self._current_user = None
    
    def retrieve_document(self, document_title: str, user: User = None) -> Document:
        """
        Retrieve a document by title if the user has permission to view it.
        
        Parameters
        ----------
        document_title : str
            The exact title of the document to retrieve
        user : User, optional
            User to check permissions for. Uses current_user if not provided.
            
        Returns
        -------
        Document or None
            Document object if found and user has permission, None otherwise
        """
        if user is None:
            user = self._current_user
            
        if user is None:
            return None
            
        doc = self._find_document_by_title(document_title)
        if doc is None:
            return None
            
        if doc.is_accessible_by(user):
            return doc
        return None
    
    def get_user_documents(self, user: User = None) -> list:
        """
        Get all documents accessible by a user.
        
        Parameters
        ----------
        user : User, optional
            User to get documents for. Uses current_user if not provided.
            
        Returns
        -------
        list
            List of Document objects accessible by the user
        """
        if user is None:
            user = self._current_user
            
        if user is None:
            return []
            
        accessible_docs = []
        for doc in self._documents:
            if doc.is_accessible_by(user):
                accessible_docs.append(doc)
        return accessible_docs
    
    def add_document(self, title: str, content: str, visibility: str = "private") -> bool:
        """
        Add a new document for the current user.
        
        Parameters
        ----------
        title : str
            Document title
        content : str
            Document content
        visibility : str
            Document visibility (default: "private")
            
        Returns
        -------
        bool
            True if document was added, False if user not logged in
        """
        if self._current_user is None:
            return False
            
        # Generate new ID (max existing ID + 1)
        new_id = max([doc.id for doc in self._documents], default=0) + 1
        
        new_doc = Document(new_id, title, content, self._current_user, visibility)
        self._documents.append(new_doc)
        return True
    
    def change_document_visibility(self, document_title: str, new_visibility: str) -> bool:
        """
        Change the visibility of a document if the user has permission.
        
        Parameters
        ----------
        document_title : str
            The exact title of the document to modify
        new_visibility : str
            The new visibility setting for the document
            
        Returns
        -------
        bool
            True if visibility was successfully changed, False otherwise
        """
        if self._current_user is None:
            return False
            
        doc = self._find_document_by_title(document_title)
        if doc is None:
            return False
            
        return doc.set_visibility(new_visibility, self._current_user)
    
    def get_all_documents(self) -> list:
        """
        Get all documents (for administrative purposes).
        
        Returns
        -------
        list
            All Document objects in the system
        """
        return self._documents.copy()  # Return copy to prevent external modification
    
    def get_all_users(self) -> list:
        """
        Get all users (for administrative purposes).
        
        Returns
        -------
        list
            All User objects in the system
        """
        return self._users.copy()  # Return copy to prevent external modification
    
    def to_dict(self) -> dict:
        """Convert manager state to dictionary for serialization."""
        return {
            "users": [user.to_dict() for user in self._users],
            "documents": [doc.to_dict() for doc in self._documents],
            "current_user": self._current_user.username if self._current_user else None
        }

In [2]:
# let's make couple of test
# Create a document manager
manager = DocumentManager()

# Login as admin
user, auth = manager.user_login("admin", "admin")
print(f"Login successful: {auth}, User: {user}")

# Retrieve a document
doc = manager.retrieve_document("Company Strategy")
print(f"Retrieved document: {doc}")

# Change document visibility
success = manager.change_document_visibility("Company Strategy", "public")
print(f"Visibility changed: {success}")

# Get all documents accessible by current user
docs = manager.get_user_documents()
print(f"Accessible documents: {len(docs)}")

# Logout
manager.user_logout()
print(f"Current user after logout: {manager.current_user}")

In [ ]:
import hashlib
import secrets

class User:
    """
    Enhanced User class with proper encapsulation.
    
    Changes:
    1. Password hashing for security
    2. Private attributes with property access
    3. Input validation
    4. Read-only properties where appropriate
    """
    
    # Class constants - define valid roles
    VALID_ROLES = ('administrator', 'user', 'manager')
    
    def __init__(self, username, role, password):
        # Use properties with validation
        self.username = username  # This uses the setter!
        self.role = role          # This uses the setter!
        
        # Private attributes (convention: start with _)
        self._password_hash = self._hash_password(password)
        self._is_active = True
        self._login_attempts = 0
    
    # =========================================================================
    # PROPERTIES - Controlled access to attributes
    # =========================================================================
    
    @property
    def username(self):
        """Get username (read-only after creation)"""
        return self._username
    
    @username.setter
    def username(self, value):
        """Set username with validation"""
        if not isinstance(value, str) or not value.strip():
            raise ValueError("Username must be a non-empty string")
        if hasattr(self, '_username'):  # Already set
            raise AttributeError("Username cannot be changed after creation")
        self._username = value.strip()
    
    @property
    def role(self):
        """Get user role"""
        return self._role
    
    @role.setter
    def role(self, value):
        """Set role with validation"""
        if value not in self.VALID_ROLES:
            raise ValueError(f"Role must be one of: {self.VALID_ROLES}")
        self._role = value
    
    @property
    def is_active(self):
        """Check if user account is active (read-only)"""
        return self._is_active
    
    @property
    def login_attempts(self):
        """Get number of failed login attempts (read-only)"""
        return self._login_attempts
    
    # =========================================================================
    # PRIVATE METHODS - Implementation details (encapsulated)
    # =========================================================================
    
    def _hash_password(self, password):
        """
        Hash password with salt for security.
        
        This is private because users don't need to know how we hash passwords.
        """
        salt = secrets.token_hex(8)  # Generate random salt
        return hashlib.sha256((password + salt).encode()).hexdigest() + ':' + salt
    
    def _verify_password(self, password):
        """
        Verify password against stored hash.
        
        This is private because password verification logic should be hidden.
        """
        try:
            hashed, salt = self._password_hash.split(':')
            return hashed == hashlib.sha256((password + salt).encode()).hexdigest()
        except ValueError:
            return False
    
    # =========================================================================
    # PUBLIC METHODS - Safe interface for external use
    # =========================================================================
    
    def authenticate(self, password):
        """
        Authenticate user with password.
        
        Now includes security features:
        - Login attempt limiting
        - Account locking after too many failures
        - Secure password verification
        """
        if not self._is_active:
            return False
        
        # Basic rate limiting
        if self._login_attempts >= 3:
            print(f"Account {self.username} temporarily locked due to too many failed attempts")
            return False
        
        if self._verify_password(password):
            self._login_attempts = 0  # Reset on successful login
            return True
        else:
            self._login_attempts += 1
            print(f"Failed login attempt #{self._login_attempts} for user {self.username}")
            return False
    
    def change_password(self, old_password, new_password):
        """
        Change user password with validation.
        
        Provides a safe way to change passwords without exposing the hash.
        """
        if not self.authenticate(old_password):
            return False
        
        if len(new_password) < 4:
            raise ValueError("Password must be at least 4 characters")
        
        self._password_hash = self._hash_password(new_password)
        return True
    
    def deactivate(self):
        """Deactivate user account - only through method"""
        self._is_active = False
    
    def activate(self):
        """Activate user account and reset login attempts"""
        self._is_active = True
        self._login_attempts = 0
    
    # =========================================================================
    # PERMISSION METHODS - Business logic encapsulation
    # =========================================================================
    
    def can_modify(self, document):
        """
        Check if user can modify a document.
        
        Business rules are encapsulated here:
        - Administrators can modify everything
        - Users can modify their own documents
        """
        if not self._is_active:
            return False
        
        if self.role == 'administrator':
            return True
        
        if document.owner == self:
            return True
        
        return False
    
    def can_view(self, document):
        """
        NEW: Check if user can view a document.
        
        We're adding this method now to prepare for better permission separation.
        """
        if not self._is_active:
            return False
        
        # Administrators can view everything
        if self.role == 'administrator':
            return True
        
        # Anyone can view public documents
        if document.visibility == 'public':
            return True
        
        # Users can view their own documents
        if document.owner == self:
            return True
        
        return False
    
    # =========================================================================
    # BACKWARD COMPATIBILITY METHODS
    # =========================================================================
    
    def to_dict(self):
        """
        Convert to dictionary for backward compatibility.
        
        Note: We return the plain password as None for security!
        We can't recover the original password from the hash.
        """
        return {
            "username": self.username,
            "role": self.role,
            "password": None  # Can't return actual password for security
        }
    
    @property
    def password(self):
        """
        Read-only property that indicates we can't reveal the password.
        
        This maintains some backward compatibility while being secure.
        """
        raise AttributeError("Password is not readable for security reasons")
    
    # =========================================================================
    # SPECIAL METHODS
    # =========================================================================
    
    def __str__(self):
        return f"User({self.username}, {self.role}, active={self.is_active})"
    
    def __repr__(self):
        return f"User('{self.username}', '{self.role}', '***')"

# =============================================================================
# TESTING THE ENCAPSULATION CHANGES
# =============================================================================

def test_encapsulation_changes():
    """
    Test the new encapsulation features while maintaining backward compatibility.
    """
    print("=" * 70)
    print("TESTING ENCAPSULATION: DATA HIDING & SECURITY")
    print("=" * 70)
    
    print("\n1. CREATING USERS WITH VALIDATION")
    print("-" * 40)
    
    try:
        # This should work
        alice = User("alice", "user", "securepassword")
        print(f"✓ Created user: {alice}")
        
        # This should fail - invalid role
        try:
            bad_user = User("bad", "invalid_role", "pass")
        except ValueError as e:
            print(f"✓ Role validation works: {e}")
        
        # This should fail - empty username
        try:
            empty_user = User("", "user", "pass")
        except ValueError as e:
            print(f"✓ Username validation works: {e}")
    
    except Exception as e:
        print(f"✗ Unexpected error: {e}")
    
    print("\n2. TESTING PASSWORD SECURITY")
    print("-" * 40)
    
    alice = User("alice", "user", "mysecret")
    
    # Test authentication
    print(f"✓ Correct password: {alice.authenticate('mysecret')}")
    print(f"✓ Wrong password: {alice.authenticate('wrong')}")
    print(f"✓ Login attempts: {alice.login_attempts}")
    
    # Test password change
    success = alice.change_password("mysecret", "newsecret")
    print(f"✓ Password change: {success}")
    print(f"✓ New password works: {alice.authenticate('newsecret')}")
    
    print("\n3. TESTING READ-ONLY PROPERTIES")
    print("-" * 40)
    
    print(f"✓ Username readable: {alice.username}")
    print(f"✓ Role readable: {alice.role}")
    print(f"✓ Active status readable: {alice.is_active}")
    
    # Try to set read-only properties
    try:
        alice.is_active = False  # Should fail
    except AttributeError as e:
        print(f"✓ Active status is read-only: {e}")
    
    # Try to read password
    try:
        _ = alice.password  # Should fail
    except AttributeError as e:
        print(f"✓ Password is not readable: {e}")
    
    print("\n4. TESTING ACCOUNT SECURITY FEATURES")
    print("-" * 40)
    
    bob = User("bob", "user", "bobpass")
    
    # Test login attempt limiting
    for i in range(5):
        bob.authenticate("wrong")
    
    print(f"✓ Account locked after too many attempts: {not bob.authenticate('bobpass')}")
    
    # Reactivate and test
    bob.activate()
    print(f"✓ Reactivation works: {bob.authenticate('bobpass')}")

if __name__ == "__main__":
    test_encapsulation_changes()